In [44]:
import pandas as pd
import numpy as np
import os
import geopandas as gpd
from dotenv import load_dotenv
load_dotenv()
import plotly.graph_objects as go
from autocensus import Query

# surpress warnings
import warnings
warnings.filterwarnings('ignore')


# set api key
MAPBOX_API = os.getenv("MAPBOX_API")
CENSUS_API = os.getenv("CENSUS_API")

### Get shapefiles
The idea is to get shapefiles by county, so that you can click on a county, and that manipulates the corresponding graphs.

In [30]:
# Configure query
query = Query(
    estimate=1,
    years=[2019],
    variables=["B03002_001E"],
    for_geo=['county:005', 'county:047','county:061','county:081','county:085'],
    in_geo=['state:36'],
    # Optional arg to add geometry: 'points', 'polygons', or None (default)
    geometry='polygons',
    # Fill in the following with your actual Census API key
    census_api_key=CENSUS_API
)

# Run query and collect output in dataframe
soql_trees = query.run()

# convert to gdf
gdf = gpd.GeoDataFrame(soql_trees)

Retrieving variables...
Retrieving ACS tables...
Retrieving shapefiles...
Merging ACS tables and variables...
Merging annotations...
Merging shapefiles...
Finalizing data...


Let's see if we can plot a basic map with population data per county.

In [53]:
gdf

,name,geo_id,geo_type,year,date,variable_code,variable_label,variable_concept,annotation,value,geometry
0,"Bronx County, New York",0500000US36005,county,2019,2019-12-31,B03002_001E,Estimate!!Total:,HISPANIC OR LATINO ORIGIN BY RACE,NaN,1418207.0,"MULTIPOLYGON (((-73.77336 40.85945, -73.77244 ..."
1,"Kings County, New York",0500000US36047,county,2019,2019-12-31,B03002_001E,Estimate!!Total:,HISPANIC OR LATINO ORIGIN BY RACE,NaN,2559903.0,"MULTIPOLYGON (((-74.04201 40.62605, -74.04199 ..."
2,"New York County, New York",0500000US36061,county,2019,2019-12-31,B03002_001E,Estimate!!Total:,HISPANIC OR LATINO ORIGIN BY RACE,NaN,1628706.0,"MULTIPOLYGON (((-74.00049 40.69876, -74.00004 ..."
3,"Queens County, New York",0500000US36081,county,2019,2019-12-31,B03002_001E,Estimate!!Total:,HISPANIC OR LATINO ORIGIN BY RACE,NaN,2253858.0,"MULTIPOLYGON (((-73.96262 40.73903, -73.96138 ..."
4,"Richmond County, New York",0500000US36085,county,2019,2019-12-31,B03002_001E,Estimate!!Total:,HISPANIC OR LATINO ORIGIN BY RACE,NaN,476143.0,"MULTIPOLYGON (((-74.16170 40.64586, -74.16060 ..."


Let's try this with plotly graph objects.

In [108]:
# Define a trace for the choropleth map
trace = go.Choroplethmapbox(geojson=gdf.geometry.__geo_interface__,
                            locations=gdf.index,
                            z=gdf["value"],
                            colorscale="Viridis",
                            zmin=0,
                            zmax=5000000,
                            marker_opacity=0.5,
                            marker_line_width=0,
                            hovertemplate="<b>%{customdata[0]}</b><br><br>" +
                                          "Population: %{z:,}<br>" +
                                          "<extra></extra>",
                            customdata=gdf[["name", "value"]])

# Define the layout for the map
layout = go.Layout(title_text="Population by County in NYC",
                   mapbox_style="mapbox://styles/mapbox/light-v10",
                   mapbox_zoom=9,
                   mapbox_center={"lat": 40.7, "lon": -73.9},
                   mapbox_accesstoken=MAPBOX_API,
                   margin={"l": 0, "r": 0, "t": 30, "b": 0})

# Create the figure with the trace and layout
fig = go.Figure(data=[trace], layout=layout)

# Show the map
fig.show()


### SoQL call
Here we will query the url and get count of:
- `tree_id` 

grouped by 

- `boroname`
- `spc_common`

We will also pivot wider:

- `health`
- `steward`

In addition, we will create percent columns for the pivoted `health` and `steward` columns.<br>
This will eliminate much of the need to wrangle in pandas.

We will get total average per `boroname` and `spc_common` for `health` and `steward` levels once in **pandas**.

In [170]:
soql_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=coalesce(spc_common, "Unknown") as spc_common,boroname,steward,count(tree_id),\
                sum(case when health = "Fair" then 1 else 0 end) as fair_health,\
                sum(case when health = "Good" then 1 else 0 end) as good_health,\
                sum(case when health = "Poor" then 1 else 0 end) as poor_health,\
                sum(case when steward = "1or2" then 1 else 0 end) as one_two_stewards,\
                sum(case when steward = "3or4" then 1 else 0 end) as three_four_stewards,\
                sum(case when steward = "4orMore" then 1 else 0 end) as four_or_more_stewards,\
                round(100 * SUM(CASE WHEN health = "Poor" THEN 1 ELSE 0 END) / COUNT(tree_id),2) AS percent_poor_health,\
                round(100 * SUM(CASE WHEN health = "Fair" THEN 1 ELSE 0 END) / COUNT(tree_id),2) AS percent_fair_health,\
                round(100 * SUM(CASE WHEN health = "Good" THEN 1 ELSE 0 END) / COUNT(tree_id),2) AS percent_good_health,\
                round(100 * SUM(CASE WHEN steward = "1or2" THEN 1 ELSE 0 END) / COUNT(tree_id),2) AS percent_one_two_stewards,\
                round(100 * SUM(CASE WHEN steward = "3or4" THEN 1 ELSE 0 END) / COUNT(tree_id),2) AS percent_three_four_stewards,\
                round(100 * SUM(CASE WHEN steward = "4orMore" THEN 1 ELSE 0 END) / COUNT(tree_id),2) AS percent_four_or_more_stewards' +\
        '&$group=spc_common,boroname,steward').replace(' ', '%20')
soql_trees= pd.read_json(soql_url)

soql_trees.head()

,spc_common,boroname,steward,count_tree_id,fair_health,good_health,poor_health,one_two_stewards,three_four_stewards,four_or_more_stewards,percent_poor_health,percent_fair_health,percent_good_health,percent_one_two_stewards,percent_three_four_stewards,percent_four_or_more_stewards
0,American beech,Bronx,1or2,6,3,2,1,6,0,0,16.67,50.00,33.33,100,0,0
1,American beech,Bronx,3or4,2,1,1,0,0,2,0,0.00,50.00,50.00,0,100,0
2,American beech,Bronx,None,23,3,18,2,0,0,0,8.70,13.04,78.26,0,0,0
3,American beech,Brooklyn,1or2,38,10,27,1,38,0,0,2.63,26.32,71.05,100,0,0
4,American beech,Brooklyn,3or4,2,0,2,0,0,2,0,0.00,0.00,100.00,0,100,0


Next step is to create the graphs for `health` and `steward` count per `spc_common` and `boroname`.
We will also want to compare against average for all, to see which is doing best.

### Question 1
What proportion of trees are in good, fair or poor health according to the `health` variable?<br>
(group by `spc_common` and `boroname`)

In [171]:
one_tree_df=soql_trees[(soql_trees['spc_common'] == "American beech") & (soql_trees['boroname'] == "Brooklyn")]

In [172]:
fig = go.Figure(data=[
    go.Bar(name='Poor', x=one_tree_df['spc_common'], y=one_tree_df['percent_poor_health'], marker_color='red'),
    go.Bar(name='Fair', x=one_tree_df['spc_common'], y=one_tree_df['percent_fair_health'], marker_color='orange'),
    go.Bar(name='Good', x=one_tree_df['spc_common'], y=one_tree_df['percent_good_health'], marker_color='green')
])

# Set the chart title and axis labels
fig.update_layout(
    title='Health Percentage Comparison by Tree Species',
    xaxis_title='Tree Species',
    yaxis_title='Health Percentage'
)

# Show the chart
fig.show()

In [163]:
species_list = soql_trees['spc_common'].unique()
species_options = [{'label': species, 'value': species} for species in species_list]

### Question 2
Are stewards (steward activity measured by the `steward` variable) having an impact on the health of trees?

We could create a grouped bar chart that groups health type by steward level. This of course would be filtered by `boroname` and `spc_common`

In [166]:
# another url call
soql_url_2 = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=coalesce(spc_common, "Unknown") as spc_common,boroname,steward,count(tree_id),\
                sum(case when health = "Fair" then 1 else 0 end) as fair_health,\
                sum(case when health = "Good" then 1 else 0 end) as good_health,\
                sum(case when health = "Poor" then 1 else 0 end) as poor_health,\
                sum(case when steward = "1or2" then 1 else 0 end) as one_two_stewards,\
                sum(case when steward = "3or4" then 1 else 0 end) as three_four_stewards,\
                sum(case when steward = "4orMore" then 1 else 0 end) as four_or_more_stewards,\
                round(100 * SUM(CASE WHEN health = "Poor" THEN 1 ELSE 0 END) / COUNT(tree_id),2) AS percent_poor_health,\
                round(100 * SUM(CASE WHEN health = "Fair" THEN 1 ELSE 0 END) / COUNT(tree_id),2) AS percent_fair_health,\
                round(100 * SUM(CASE WHEN health = "Good" THEN 1 ELSE 0 END) / COUNT(tree_id),2) AS percent_good_health,\
                round(100 * SUM(CASE WHEN steward = "1or2" THEN 1 ELSE 0 END) / COUNT(tree_id),2) AS percent_one_two_stewards,\
                round(100 * SUM(CASE WHEN steward = "3or4" THEN 1 ELSE 0 END) / COUNT(tree_id),2) AS percent_three_four_stewards,\
                round(100 * SUM(CASE WHEN steward = "4orMore" THEN 1 ELSE 0 END) / COUNT(tree_id),2) AS percent_four_or_more_stewards' +\
        '&$group=spc_common,boroname,steward').replace(' ', '%20')
soql_trees_2= pd.read_json(soql_url_2)

soql_trees_2.head()

,spc_common,boroname,steward,count_tree_id,fair_health,good_health,poor_health,one_two_stewards,three_four_stewards,four_or_more_stewards,percent_poor_health,percent_fair_health,percent_good_health,percent_one_two_stewards,percent_three_four_stewards,percent_four_or_more_stewards
0,American beech,Bronx,1or2,6,3,2,1,6,0,0,16.67,50.00,33.33,100,0,0
1,American beech,Bronx,3or4,2,1,1,0,0,2,0,0.00,50.00,50.00,0,100,0
2,American beech,Bronx,None,23,3,18,2,0,0,0,8.70,13.04,78.26,0,0,0
3,American beech,Brooklyn,1or2,38,10,27,1,38,0,0,2.63,26.32,71.05,100,0,0
4,American beech,Brooklyn,3or4,2,0,2,0,0,2,0,0.00,0.00,100.00,0,100,0


In [177]:
# boro and tree
boro="Brooklyn"
tree="American beech"

# filter by tree type and boro
one_tree_df_2=soql_trees_2[(soql_trees_2['spc_common'] == tree) & (soql_trees_2['boroname'] == boro)]

# Create the bar chart
fig = go.Figure()

# Add the "Poor" health status bar
fig.add_trace(
    go.Bar(
        x=one_tree_df_2['steward'].unique(),
        y=one_tree_df_2['percent_poor_health'],
        name='Poor',
        marker_color='red'
    )
)

# Add the "Fair" health status bar
fig.add_trace(
    go.Bar(
        x=one_tree_df_2['steward'].unique(),
        y=one_tree_df_2['percent_fair_health'],
        name='Fair',
        marker_color='orange'
    )
)

# Add the "Good" health status bar
fig.add_trace(
    go.Bar(
        x=one_tree_df_2['steward'].unique(),
        y=one_tree_df_2['percent_good_health'],
        name='Good',
        marker_color='green'
    )
)

# Update the layout
fig.update_layout(
    title=f'Health Status of {tree} Percentage by Steward Type in {boro}',
    xaxis_title='Steward Type',
    yaxis_title='Percentage (%)',
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1,
    legend=dict(
        x=0,
        y=1.15,
        orientation='h'
    )
)

# Show the plot
fig.show()
